# Loop Workflow Example 2

> * High level approach to making a 3D model from just a bounding box you can draw
> * To run this notebook for the first time, there are some dependencies needed to use the interactive map. Execute the cell immediately below and restart jupyter.

In [ ]:
#if not already installed:
#!conda install -c loop3d map2loop loopstructural pyamg meshio
#!conda install -c conda-forge leafmap ipyleaflet ipywidgets jupyter-ipywidgets jupyter-nbextension -y
# and
#!pip install geoh5py
# may need to restart jupyter server
import warnings
warnings.filterwarnings('ignore')

## Leaflet Maps

In [ ]:
import ipywidgets as widgets
import os
import matplotlib.pyplot as plt

# load last saved map area and model engine (if they exist)
if(not os.path.isdir('../scratch/')):
    os.mkdir('../scratch/')
if(os.path.isfile('../scratch/last_choices.txt')):
    f=open('../scratch/last_choices.txt','r')
    contents =f.readlines()
    f.close()
    default_map=contents[0].replace("\n","")
    default_engine=contents[1].replace("\n","")
else:
    default_map='Turner_Syncline'
    default_engine='loopstructural'

options=['Draw Your Own','Last Area Drawn']

if(not default_map in options):
    default_map= options[0]

map_choice=widgets.Dropdown(
    options=options,
    value=default_map,
    description='Map area:',
    disabled=False,
)
display(map_choice)

In [ ]:
test_data_name=map_choice.value
print(test_data_name)

In [ ]:
import leafmap
import pandas as pd
from shapely.geometry import Polygon
import geopandas as gpd
src_crs = "epsg:4326"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)
dst_crs = "epsg:28350"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)

if(not test_data_name =='Draw Your Own'):
    if(test_data_name=='Last Area Drawn'):
        last_coords=pd.read_csv('../scratch/last_area.csv')
        display(last_coords)
        minx=last_coords.iloc[0]['minx']
        miny=last_coords.iloc[0]['miny']
        maxx=last_coords.iloc[0]['maxx']
        maxy=last_coords.iloc[0]['maxy']
    elif(not test_data_name =='Draw Your Own'):
        y_point_list = [miny, miny, maxy, maxy, maxy]
        x_point_list = [minx, maxx, maxx, minx, minx]
        bbox_geom = Polygon(zip(x_point_list, y_point_list))
        polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom])
        polygon_ll=polygon.to_crs(src_crs)

        minx=polygon_ll.total_bounds[0]
        maxx=polygon_ll.total_bounds[2]
        miny=polygon_ll.total_bounds[1]
        maxy=polygon_ll.total_bounds[3]

        minlong=minx
        maxlong=maxx
        minlat=miny
        maxlat=maxy
        #print("x",polygon_ll.total_bounds[0])
        st_bbox=[minlong,minlat,maxlong,maxlat]
        lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
        lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
        bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
        rect = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 

    bbox2=str(minx)+","+str(miny)+","+str(maxx)+","+str(maxy)
    y_point_list = [miny, miny, maxy, maxy, maxy]
    x_point_list = [minx, maxx, maxx, minx, minx]
    bbox_geom = Polygon(zip(x_point_list, y_point_list))
    polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom])
    polygon_ll=polygon.to_crs(src_crs)

    minlong=polygon_ll.total_bounds[0]
    maxlong=polygon_ll.total_bounds[2]
    minlat=polygon_ll.total_bounds[1]
    maxlat=polygon_ll.total_bounds[3]
    
    minlong=minx
    maxlong=maxx
    minlat=miny
    maxlat=maxy

    lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
    lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
    bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
    rect = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 

    center=(minlat+((maxlat-minlat)/2),minlong+((maxlong-minlong)/2))
else:
    center=(-22.6,117.3)

m = leafmap.Map( center=center, zoom=8,scroll_wheel_zoom=True)
m.add_basemap( basemap='OpenTopoMap')
m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='2_5m_interpgeop15_4326',
    format='image/png',
    transparent=True,
    opacity=0.4,
    attribution='Geology data from GSWA',
    name='geology',
    shown=True)
# url = 'https://www.loopwms.xyz/geoserver/loop/wms?'
m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='2_5m_interpstrucl15_4326',
    format='image/png',
    transparent=True,
    opacity=0.4,
    attribution='Linear features data from GSWA',
    name='faults/folds')
m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='waroxi_wa_4326_bed',
    format='image/png',
    transparent=True,
    attribution='Outcrop data from GSWA',
    name='outcrops')



if( not test_data_name =='Draw Your Own'):
    m.add_gdf(rect, layer_name="Last Rectangle", 
                   fill_colors=['orange'])

display(m)

In [ ]:
polyclip=False # implemented in future version

if(test_data_name=='Draw Your Own' or test_data_name=='Last Area Drawn'):
    if(test_data_name=='Draw Your Own'):
        src_crs = "epsg:4326"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)
        dst_crs = "epsg:28350" # coordinate system for example data
        
        rect=m.draw_features

        if(len(rect)==0):
            minlong=117.13698
            maxlong=117.564464
            minlat= -22.690712
            maxlat=-22.396454

        else:
            minlong=rect[0]['geometry']['coordinates'][0][0][0]
            maxlong=rect[0]['geometry']['coordinates'][0][2][0]
            minlat= rect[0]['geometry']['coordinates'][0][0][1]
            maxlat= rect[0]['geometry']['coordinates'][0][1][1]    
    else:
        use_roi_clip=False
        roi_clip_path=''

    bounds=(minlong,maxlong,minlat,maxlat)

    lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
    lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
    bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
    mbbox = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 
    bbox=mbbox.total_bounds
    st_bbox=[bbox[0],bbox[1],bbox[2],bbox[3]]
    print(src_crs,mbbox.total_bounds)
    mbbox=mbbox.to_crs(dst_crs)
    print(dst_crs,mbbox.total_bounds)
    
    f=open('../scratch/last_area.csv','w') 
    ostr='minx,miny,maxx,maxy\n'
    f.write(ostr)
    ostr=str(minlong)+','+str(minlat)+','+str(maxlong)+','+str(maxlat)+'\n'
    f.write(ostr)
    f.close()
    

## Map2Loop

In [ ]:
import os
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel
from map2loop.thickness_calculator import StructuralPoint
from datetime import datetime

nowtime=datetime.now().isoformat(timespec='minutes')
bbox_3d={
    "minx": mbbox.total_bounds[0], #500000,
    "miny": mbbox.total_bounds[1], #7490000,
    "maxx": mbbox.total_bounds[2], #545000,
    "maxy": mbbox.total_bounds[3], #7520000,
    "base": -4800,
    "top": 1200,
}
model_name=os.path.join(".",nowtime.replace(":","-").replace("T","-"))
loop_project_filename=os.path.join(model_name, "wa_output.loop3d")
proj = Project(
    use_australian_state_data = "WA",
    verbose_level = VerboseLevel.NONE,
    tmp_path  = model_name,
    working_projection = "EPSG:28350",
    bounding_box = bbox_3d,
    loop_project_filename = loop_project_filename,
    overwrite_loopprojectfile = True
)

proj.set_thickness_calculator(StructuralPoint())
proj.run_all(take_best=True)

In [ ]:
# Draw overlay of point data on geology map (can also try 'contacts','orientations','faults')
proj.draw_geology_map(overlay="contacts")

In [ ]:
import LoopProjectFile as LPF
loop_project_filename="wa_output.loop3d"
projFile = LPF.ProjectFile(loop_project_filename)
projFile.stratigraphicLog.ThicknessMedian

projFile.stratigraphicLog

## Loop Structural

In [ ]:
# Define project pathing from m2l
dtm_path=model_name+'/dtm/'
vtk_path=model_name+'/vtk/'
if not os.path.exists(vtk_path):
    os.mkdir(vtk_path)
filename=vtk_path+'/'+'surface_name_{}.vtk'

bounding_box = proj.map_data.get_bounding_box()
minx = bounding_box['minx']
maxx = bounding_box['maxx']
miny = bounding_box['miny']
maxy = bounding_box['maxy']
model_base = bounding_box['base']
model_top = bounding_box['top']

In [ ]:
import LoopProjectFile as LPF
import LoopStructural
from LoopStructural.modelling.input.project_file import LoopProjectfileProcessor as LPFProcessor

projFile = LPF.ProjectFile(loop_project_filename)
processedData = LPFProcessor(projFile)
model = LoopStructural.GeologicalModel.from_processor(processedData)
model.update()

In [ ]:
# Create viewer for Loop Structural output
from LoopStructural.visualisation import Loop3DView
view = Loop3DView(model, window_size=[1200,800])
view.plot_model_surfaces(fault_colour="red")

# Add dtm method not currently implimented but will be coming soon
#view.add_dtm()

# Set inital camera position and orientation
view.camera_position = "xz"
view.camera.elevation = 40

view.show(jupyter_backend='client')

In [ ]:
save_vtk=True

surface_verts = {}

def function(xyz,tri,name): # for saving out vtk files
    xyz = np.copy(xyz)
    tri = np.copy(tri)
    nanmask = np.all(~np.isnan(xyz),axis=1)
    vert_idx = np.zeros(xyz.shape[0],dtype=int) -1
    vert_idx[nanmask] = np.arange(np.sum(nanmask))
    tri[:] = vert_idx[tri]
    tri = tri[np.all(tri > -1,axis=1),:]
    xyz = xyz[nanmask,:]
    surface_verts[name] = (xyz,tri)

if(save_vtk):
    from LoopStructural.visualisation.vtk_exporter import VtkExporter
    view = VtkExporter(model,vtk_path)

    view.nsteps = np.array([200,200,200])

    view.nelements = 1e5
    view.add_model_surfaces(function=function,filename=filename,faults=False)
    view.nelements=1e6
    view.add_model_surfaces(function=function,filename=filename,strati=False,displacement_cmap = 'rainbow')

## Save voxel model

In [ ]:
save_voxels=True
if (save_voxels):
    voxel_size = 500
    sizex = int((maxx - minx)/voxel_size)
    sizey = int((maxy - miny)/voxel_size)
    sizez = int((model_top - model_base)/voxel_size)
    print('voxel_size=', voxel_size, 
          ', saved in Z,Y,X order 16 bit unisgned, X(height)=', sizex, 
          ', Y(#ims)=', sizey, ', Z(width)=', sizez)
    print('lower south west corner: west=', minx, ', south=', miny, ', lower=', model_base)
    voxels = model.evaluate_model(model.regular_grid(nsteps = (sizey, sizex, sizez), shuffle = False), scale = False)
    voxels.astype('int16').tofile(model_name + '/vtk/voxels.raw')
    print('voxels saved as', model_name + '/vtk/voxels.raw')

## Save Geoscience Analyst Model

In [ ]:
#code to parse a directory of .obj files and combine them into
#a single geoh5 file that GeoscienceAnalyst can read
#Requires installation of https://github.com/MiraGeoscience/geoh5py 
#(note .geoh5 meshes are 0 based)
#Run this code after obj files have been created (see Example notebooks)

import pandas as pd
from os import listdir
import os
from os.path import isfile, join
from pathlib import Path
import numpy as np
from geoh5py.objects import BlockModel
from geoh5py.workspace import Workspace
from geoh5py.objects import Surface

def hextofloats(h):
    '''Takes a hex rgb string (e.g. #ffffff) and returns an RGB tuple (float, float, float).'''
    return tuple(int(h[i:i + 2], 16) / 255. for i in (1, 3, 5))  # skip '#'

def geoh5_create_surface_data(obj_path_dir,colour_path):

    h5file_path = obj_path_dir+"/loop.geoh5"

    workspace = Workspace(h5file_path)
    onlyfiles = [f for f in listdir(obj_path_dir) if isfile(join(obj_path_dir, f))]
    colour_index=0
    units=proj.stratigraphic_column.stratigraphicUnits.copy().set_index("name")
    colour_map=open(obj_path_dir+'/loop_colour_map.clr','w')
    colour_map.write('{\tStart\tRed\tGreen\tBlue\t}\n')

    for file in onlyfiles:
        if ('.obj' in file):
            obj=pd.read_csv(obj_path_dir+'/'+file,' ',names=["code","X","Y","Z"])
            indices=obj[obj['code']=='f']
            vertices=obj[obj['code']=='v']
            vertices=vertices.drop(['code'], axis=1)
            indices=indices[list("XYZ")].astype(int)
            i=indices.to_numpy()-1
            v=vertices.to_numpy()
            if(len(i)>0 and len(v)>0):
                # Create a geoh5 surface
                surface = Surface.create(
                          workspace, name=file.replace('.obj',''), vertices=v, cells=i
                            )
                if('Fault_' in file or 'dtm' in file):
                    colours=np.ones(surface.n_cells)*99
                else:
                    colours=np.ones(surface.n_cells)*colour_index
                    rgb=hextofloats(units.loc[file.replace('.obj','')]['colour'])
                    colour_map.write('{}\t{}\t{}\t{}\n'.format(colour_index,rgb[0],rgb[1],rgb[2]))
                    colour_index=colour_index+1
                    

                surface.add_data({
                    "colour_index": {
                        "association":"CELL",
                        "values": colours
                    }
                })
                
                workspace.save_entity(surface)
    workspace.close()
                
    colour_map.write('{}\t{}\t{}\t{}\n'.format(99,1,1,1))
    colour_map.close()
    print("colour map saved as:",obj_path_dir+'/loop_colour_map.clr')


save_GA=False

if(save_GA):
    geoh5_create_surface_data(vtk_path,proj.map_data.tmp_path)
